In [1]:
import numpy as np
from fluids import *


In [2]:
PM_OR = 0.049 # PM outer radius [m]
PM_IR = 0.045 # PM inner radius [m]
U_IR = 0.03 # Uranium layer inner radius [m]
U_rho = 8550 # Uranium mixture density [kg/m^3]

PM_thick = PM_OR - PM_IR # PM thickness [m]
L = 0.81 # CFE length [m]

# Define operating conditions
w = 7000 * 2*np.pi/60 # CFE rotation speed [rad/s]

d = np.load("density_500CEA.npy")
r = np.load("radius_500CEA.npy")
dPs = []
fact = w**2/(PM_IR)
for i in range(r.shape[0]):
    dP = r[i]**2 * d[i]
    dPs.append(dP)

# P_centripetal = (2/3)*w**2*np.pi*U_rho*(PM_IR**3 - U_IR**3) / (2*np.pi*PM_IR) # Centripetal pressure from Uranium
P_centripetal = - np.sum(dPs) * np.diff(r)[0] * fact

P_out = 10e6 + P_centripetal # Outlet pressure [MPa]
f = H2(T=450, P=P_out) # Fluid state

mflow = 0.1 # Mass flow rate [kg/s]
rho = f.rho # Fluid density [kg/m^3]
Q = mflow / rho # Volumetric flow rate [m^3/s]
A = 2*np.pi*PM_OR*L # Cross sectional area [m^2]
q = Q / A # Darcian velocity [m/s]
mu = f.mu # Fluid viscosity [Pa * s]

k1 = 5.69771546674471E-12 # for 18% open porosity, 35% total
k2 = np.exp(-1.71588/(k1**0.08093)) # Correlation from Dey et al.

P_inlet = np.sqrt((mu*q/k1 + (rho*q**2)/k2) * 2*P_out*PM_thick + P_out**2) # Inlet pressure to PM [Pa]

PM_dP =  P_inlet - P_out # Pressure drop across PM [Pa]


print(f"T = 450K, P = 12.5 MPa\n")
print(f"rho = {rho}")
print(f"Q = {Q}")
print(f"A = {A}")
print(f"q = {q}")
print(f"mu = {mu}")
print(f"k1 = {k1}")
print(f"k2 = {k2}")
print(f"P_chamber = {P_out}")
print(f"P_inlet = {P_inlet}")
print(f"PM_dP = {PM_dP}")

T = 450K, P = 12.5 MPa

rho = 6.389293864533732
Q = 0.01565118182387713
A = 0.2493796248419578
q = 0.06276046743512399
mu = 1.1895048408744165e-05
k1 = 5.69771546674471e-12
k2 = 8.765844420214738e-07
P_chamber = 13112903.904680518
P_inlet = 13113542.825584149
PM_dP = 638.9209036305547


Plotting

In [ ]:
from fluids import H2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='Times New Roman',size="10")
mpl.rc('figure', figsize=(4.8,3.6))
mpl.rc('savefig', dpi=800)
mpl.rc('lines', linewidth=1.2)
mpl.rc('axes', grid=True)
mpl.rc('grid', linewidth=0.25)
mpl.rc('mathtext', fontset="dejavuserif")
mpl.rc('xtick.minor', visible=True, size=1.5, width=0.5)
mpl.rc('ytick.minor', visible=True, size=1.5, width=0.5)
plt.rcParams['figure.constrained_layout.use'] =  True

base = {"P_core":10e6,
        "T_channel":450,
        "r5":56e-3,
        "d56":8e-3,
        "N":7000,
        "nu_s":0.691,
        "L_CFE":.84,
        "T_core":3700}

stdlim = [0.5, 2]

# ******************************************
# TODO: if you don't want to plot vs a particular parameter, remove it from `vary`
# TODO: if you want to plot a particular parameter over a range different from others, replace its limits in `vary`
# ******************************************

vary = {"P_core":stdlim,
        "T_channel":stdlim,
        # "r5":stdlim,
        # "d56":[0.125, 2],
        "N":stdlim,
        # "nu_s":stdlim,
        "L_CFE":stdlim}

labels = {"P_core":"core pressure $P_3$", 
        "T_channel":"channel temperature $T_1$",
        "r5":"case radius $r_5$",
        "d56":"outer channel width $(r_6 - r_5)$",
        "N":"CFE rotation rate $\omega$",
        "nu_s":"turbine sp. speed $\\nu_s$",
        "L_CFE":"CFE length $l$"}

yvals = dict()
xvals = dict()

base_core = H2(P=base["P_core"], T=base["T_core"])      # speed code up by not calculating on every single loop

for key in vary:                    # iterate through properties we want to vary
    props = base.copy()             # reset all properties to base values
    lim = vary[key]                 # relative property value limits
    n_pts = 50                      # number of x-value points to plot to form a smooth curve
    xvals[key] = np.arange(lim[0], lim[1]+1e-5, np.diff(lim)/n_pts)
    yvals[key] = []
    for x in xvals[key]:
        props[key] = x * base[key]              # adjust single parameter
        L_total = props["L_CFE"] + 0.1          # compute total length
        if key=="P_core" or key=="T_core":      # check if core state needs adjustment
            core = H2(P=props["P_core"], T=props["T_core"])
        else:
            core = base_core
        w = props["N"] * np.pi/30           # compute omega
        r6 = props["r5"] + props["d56"]         # compute r6


        # ******************************************
        y=1; print("CHANGE THIS LINE!")
        # TODO: ADD YOUR CALCULATIONS FOR THE OUTPUT Y-AXIS VALUE BASED ON INPUTS
        # TODO: SET y=<YOUR Y-VALUE PROPERTY>
        # ******************************************

        d = np.load("density_500CEA.npy")
        r = np.load("radius_500CEA.npy")
        dPs = []
        fact = w**2/(PM_IR)
        for i in range(r.shape[0]):
            dP = r[i]**2 * d[i]
            dPs.append(dP)

        # P_centripetal = (2/3)*w**2*np.pi*U_rho*(PM_IR**3 - U_IR**3) / (2*np.pi*PM_IR) # Centripetal pressure from Uranium
        P_centripetal = - np.sum(dPs) * np.diff(r)[0] * fact

        P_out = props["P_core"] + P_centripetal # Outlet pressure [MPa]
        f = H2(T=props["T_channel"], P=P_out) # Fluid state

        mflow = 0.108 # Mass flow rate [kg/s]
        rho = f.rho # Fluid density [kg/m^3]
        Q = mflow / rho # Volumetric flow rate [m^3/s]
        A = 2*np.pi*PM_OR*props["L_CFE"] # Cross sectional area [m^2]
        q = Q / A # Darcian velocity [m/s]
        mu = f.mu # Fluid viscosity [Pa * s]

        k1 = 5.69771546674471E-12 # for 18% open porosity, 35% total
        k2 = np.exp(-1.71588/(k1**0.08093)) # Correlation from Dey et al.

        P_inlet = np.sqrt((mu*q/k1 + (rho*q**2)/k2) * 2*P_out*PM_thick + P_out**2) # Inlet pressure to PM [Pa]

        PM_dP =  P_inlet - P_out # Pressure drop across PM [Pa]
        y = P_inlet
        yvals[key].append(y)
np.savez("PM_PressureDrop/PM_drop_params.npz", xvals)
np.savez("PM_PressureDrop/PM_drop.npz", yvals)



for key in yvals:       # plot each line
    plt.plot(yvals[key], label=labels[key])

plt.legend()
plt.xlim(0,2.2)

# ******************************************
# TODO: update figure name
# ******************************************

plt.savefig("FIGURENAME.svg")
plt.show()



        